# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import ResNet38 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CNN_finetune/config.yaml'

In [7]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [8]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # model out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['param']['sample_rate'],
                window_size=config['param']['window_size'],
                hop_size=config['param']['hop_size'],
                mel_bins=config['param']['mel_bins'],
                fmin=config['param']['fmin'],
                fmax=config['param']['fmax'],
                classes_num=6
                )
    optimizer = optim.Adam(net.parameters())
    criterion = nn.CrossEntropyLoss()
    num_epochs = config['param']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # load pre-trained model
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_ROOT'])
    net.load_state_dict(pretrained_dict['model'], strict=False)
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, criterion, scheduler, num_epochs, writer, model_out_path)
    
    com.toc()

In [10]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [ ]:
for machine_type in machine_types[5:]:
    run(machine_type, dev_paths)

2021-04-09 17:09:45,520 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyCar
2021-04-09 17:09:45,536 - 00_train.py - INFO - MAKE DATA_LOADER
2021-04-09 17:09:45,539 - 00_train.py - INFO - TRAINING
2021-04-09 17:09:52,007 - pytorch_modeler.py - INFO - train
  0%|          | 0/189 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 189/189 [03:45<00:00,  1.19s/it]
2021-04-09 17:13:37,759 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:09<00:00,  2.03it/s]
2021-04-09 17:13:50,365 - pytorch_modeler.py - INFO - Save best model
2021-04-09 17:13:50,368 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:09<00:00,  2.05it/s]
2021-04-09 17:13:59,704 - pytorch_modeler.py - INFO - epoch:1/40, train_losses:21.375534176826477, val_source_losses:0.925961, val_target_losses:1.125028, best_flag:True
2021-04-09 17:13:59,708 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:43<00:00,  1.18s/it]
2021-04-09 17:17:42,831 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:09<00:00,  2.10it/s]
2021-04-09 17:17:55,137 - pytorch_modeler.py - INFO - Save best model
2021-04-09 17:17:55,138 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:09<00:00,  2.07it/s]
2021-04-09 17:18:04,328 - pytorch_modeler.py - INFO - epoch:2/40, trai

100%|██████████| 189/189 [03:59<00:00,  1.27s/it]
2021-04-09 18:16:56,602 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.43it/s]
2021-04-09 18:17:04,477 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:07<00:00,  2.45it/s]
2021-04-09 18:17:12,228 - pytorch_modeler.py - INFO - epoch:16/40, train_losses:5.112396653974429, val_source_losses:0.279979, val_target_losses:0.269074, best_flag:False
2021-04-09 18:17:12,229 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [04:08<00:00,  1.31s/it]
2021-04-09 18:21:20,605 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:08<00:00,  2.31it/s]
2021-04-09 18:21:29,042 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:08<00:00,  2.18it/s]
2021-04-09 18:21:37,777 - pytorch_modeler.py - INFO - epoch:17/40, train_losses:3.8862700816971483, val_source_losses:0.358471, val_target_losses:0.204541, best_flag:False
2021-04-09 18:21:37,778 - pytorch_mod

2021-04-09 19:23:50,115 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [04:03<00:00,  1.29s/it]
2021-04-09 19:27:53,671 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.50it/s]
2021-04-09 19:28:01,341 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:07<00:00,  2.52it/s]
2021-04-09 19:28:08,899 - pytorch_modeler.py - INFO - epoch:32/40, train_losses:3.8725493722247393, val_source_losses:0.303066, val_target_losses:0.203818, best_flag:False
2021-04-09 19:28:08,900 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [04:07<00:00,  1.31s/it]
2021-04-09 19:32:16,709 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.54it/s]
2021-04-09 19:32:24,226 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:07<00:00,  2.48it/s]
2021-04-09 19:32:31,882 - pytorch_modeler.py - INFO - epoch:33/40, train_losses:3.043847521736211, val_source_losses:0.250508, val_target_losses:0.1

elapsed time: 10457.254244328 [sec]


2021-04-09 20:04:05,081 - pytorch_modeler.py - INFO - train
  0%|          | 0/189 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 189/189 [03:54<00:00,  1.24s/it]
2021-04-09 20:07:59,211 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.66it/s]
2021-04-09 20:08:09,229 - pytorch_modeler.py - INFO - Save best model
2021-04-09 20:08:09,299 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:07<00:00,  2.64it/s]
2021-04-09 20:08:16,510 - pytorch_modeler.py - INFO - epoch:1/40, train_losses:19.94741467759013, val_source_losses:0.265514, val_target_losses:1.049864, best_flag:True
2021-04-09 20:08:16,528 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:54<00:00,  1.24s/it]
2021-04-09 20:12:11,421 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.67it/s]
2021-04-09 20:12:21,488 - pytorch_modeler.py - INFO - Save best model
2021-04-09 20:12:21,604 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:07<00:00,  2.62it/s]
2021-04-09 20:12:28,849 - pytorch_modeler.py - INFO - epoch:2/40, train

2021-04-09 21:12:35,723 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [04:03<00:00,  1.29s/it]
2021-04-09 21:16:39,324 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.52it/s]
2021-04-09 21:16:46,877 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:07<00:00,  2.62it/s]
2021-04-09 21:16:54,182 - pytorch_modeler.py - INFO - epoch:17/40, train_losses:3.4250811519400335, val_source_losses:0.656997, val_target_losses:0.180267, best_flag:False
2021-04-09 21:16:54,183 - pytorch_modeler.py - INFO - train
 44%|████▍     | 84/189 [02:35<12:06,  6.92s/it]

In [ ]:
machine_type = 'pump'
input_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
ext_data = pd.read_pickle(input_path)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['valid_source']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)

In [ ]:
print(M_means.shape)
plt.imshow(M_means, aspect='auto', cmap='jet')
plt.title(phase)
plt.colorbar()
plt.show()